In [1]:
#load the original data
import pickle
with open('./train_data', 'rb') as f:
    train_data = pickle.load(f)
    train_label= pickle.load(f)
with open('./test_data', 'rb') as f:
    test_data = pickle.load(f)

In [2]:
import numpy as np
import tensorflow as tf
import random as rn

In [3]:
#transform the data to correct image form

#train data
a=[]
for i in range(50000):
    single_img = np.array(train_data[i])
    single_img_reshaped = np.transpose(np.reshape(single_img,(3, 32,32)), (1,2,0))
    a.append(single_img_reshaped)
train_data=np.array(a)

#train label
train_label=np.array(train_label).reshape(50000,1)

#test data
b=[]
for i in range(10000):
    single_img = np.array(test_data[i])
    single_img_reshaped = np.transpose(np.reshape(single_img,(3, 32,32)), (1,2,0))
    b.append(single_img_reshaped)
test_data=np.array(b)

print(train_data.shape,test_data.shape,train_label.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 1)


In [4]:
import matplotlib.pyplot as plt

t=train_data/255.0

# show the data through plot
plt.figure()  # create new figure
fig_size = [20, 20]  # specify figure size
plt.rcParams["figure.figsize"] = fig_size  # set figure size

# Plot first 100 train image of dataset
for i in range(1, 101):
    ax = plt.subplot(10, 10, i)  # Specify the i'th subplot of a 10*10 grid
    img = t[i, :, :, :]  # Choose i'th image from train data
    ax.get_xaxis().set_visible(False)  # Disable plot axis.
    ax.get_yaxis().set_visible(False)
    plt.imshow(img)

#plt.show()

In [5]:
# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.

import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(8)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(80)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

In [6]:
from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(800)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [7]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout
from keras.layers import Flatten, Dense, Activation
from keras import optimizers
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import math
from scipy.stats import binom
import scipy

In [8]:
class VGG16_CIFAR100:
    
    def __init__(self):
        self.num_classes = 100
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]
        self.batch_size = 128
        self.epoches = 100
        self.learning_rate = 0.1
        self.lr_decay = 1e-6
        
    # Function to create dataset for training and validation of model
    def create_dataset(self): 
        
        num_classes = self.num_classes
        
        # Create Train and Test datasets:
        train_dat=train_data.reshape((train_data.shape[0],3072)) #reshape to(50000,3072)
        
        train_dat_df=pd.DataFrame(train_dat) #transform to dataframe
        train_label_df=pd.DataFrame(train_label)
        
        x_train_df=train_dat_df.sample(frac=0.9,axis=0) #use 90% in training set as x_train
        y_train_df=train_label_df.iloc[list(x_train_df.index)]
        
        bad_df = train_dat_df.index.isin(list(x_train_df.index))
        x_test_df=train_dat_df[~bad_df] # remaining 10% as x_test
        y_test_df=train_label_df.iloc[list(x_test_df.index)]
        
        x_train=np.array(x_train_df).reshape((np.array(x_train_df).shape[0],32,32,3)).astype('float32')
        x_test =np.array(x_test_df).reshape((np.array(x_test_df).shape[0],32,32,3)).astype('float32')
        y_train=np.array(y_train_df).reshape((np.array(x_train_df).shape[0],1))
        y_test=np.array(y_test_df).reshape((np.array(x_test_df).shape[0],1))
        
        x_test_p=test_data.astype('float32') #test set that needs to be predicted
        
        # Normalize the data
        x_train, x_test, x_test_p= self.normalize(x_train, x_test, x_test_p)
        
        # Create one-hot encodings
        y_train = np_utils.to_categorical(y_train, num_classes)
        y_test = np_utils.to_categorical(y_test, num_classes)
        
        return x_train, y_train, x_test, y_test, x_test_p
    
    # Function to normalize train and validation datasets
    def normalize(self,X_train, X_test, X_test_p): 
        
        # Compute Mean
        mean = np.mean(X_train,axis=(0, 1, 2, 3))
        
        # Compute Standard Deviation
        std = np.std(X_train, axis=(0, 1, 2, 3)) 
        
        # Normalize the data
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        X_test_p = (X_test_p-mean)/(std+1e-7)
        
        return X_train, X_test, X_test_p
    
    def buildmodel(self): 
        
        weight_decay = self.weight_decay
        num_classes = self.num_classes
        x_shape = self.x_shape
        
        model = Sequential()
        
        # First group of convolutional layer
        model.add(Conv2D(64, (3, 3), padding='same', input_shape = x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Second group of convolutional layer
        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Third group of convolutional layer
        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Fourth group of convolutional layer
        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Fifth group of convolutional layer
        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Two Fully connected layer
        model.add(Flatten())
        model.add(Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        
        return model
    
    def model_train(self, model, x_train, y_train, x_test, y_test):
        
        # Training parameters
        batch_size = self.batch_size
        number_epoches = self.epoches
        learning_rate = self.learning_rate
        lr_decay = self.lr_decay

            # Data augmentation
        dataaugmentation = ImageDataGenerator(
                                featurewise_center=False,  # set input mean to 0 over the dataset
                                samplewise_center=False,  # set each sample mean to 0
                                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                samplewise_std_normalization=False,  # divide each input by its std
                                zca_whitening=False,  # apply ZCA whitening
                                rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                horizontal_flip=True,  # randomly flip images
                                vertical_flip=False)  # randomly flip images
        
        dataaugmentation.fit(x_train)
            
        # Optimization details
        sgd = optimizers.SGD(lr=0.0, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # Function to reduce learning rate by half after every 25 epochs
        def step_decay(epoch):
            # LearningRate = InitialLearningRate * DropRate^floor(Epoch / EpochDrop)
        
            initial_lrate = 0.1
            drop = 0.5
            epochs_drop = 25.0
            lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
            return lrate

        # Callback for learning rate schedule
        lrate = LearningRateScheduler(step_decay)
        callbacks_list = [lrate]

        # spe = Steps per epoch
        spe = x_train.shape[0] // batch_size
        
        # Fit the model
        model.fit_generator(dataaugmentation.flow(x_train, y_train,
                                                  batch_size = batch_size),
                            steps_per_epoch = spe, callbacks=callbacks_list,
                            epochs = number_epoches,
                            validation_data = (x_test, y_test))
        
        return model

In [ ]:
# Create class object
model_cifar100 = VGG16_CIFAR100()

# Training and validation datasets
x_train, y_train, x_test, y_test, x_test_p = model_cifar100.create_dataset()

# Create model
model = model_cifar100.buildmodel()

# Train the model
model = model_cifar100.model_train(model, x_train, y_train, x_test, y_test)

Epoch 1/100
351/351 [==============================] - 2483s 7s/step - loss: 22.4789 - acc: 0.0279 - val_loss: 23.9973 - val_acc: 0.0084
Epoch 2/100
351/351 [==============================] - 2433s 7s/step - loss: 13.7472 - acc: 0.0422 - val_loss: 11.3182 - val_acc: 0.0150
Epoch 3/100
351/351 [==============================] - 2442s 7s/step - loss: 9.3732 - acc: 0.0502 - val_loss: 8.1251 - val_acc: 0.0318
Epoch 4/100
351/351 [==============================] - 2432s 7s/step - loss: 7.0046 - acc: 0.0559 - val_loss: 8.6841 - val_acc: 0.0204
Epoch 5/100
351/351 [==============================] - 2434s 7s/step - loss: 6.5605 - acc: 0.0350 - val_loss: 6.3180 - val_acc: 0.0226
Epoch 6/100
351/351 [==============================] - 2437s 7s/step - loss: 5.6709 - acc: 0.0495 - val_loss: 11.6307 - val_acc: 0.0134
Epoch 7/100
351/351 [==============================] - 2437s 7s/step - loss: 4.8399 - acc: 0.0686 - val_loss: 11.1310 - val_acc: 0.0194
Epoch 8/100
351/351 [============================

In [ ]:
# Prediction on test set
predict_test = model.predict(x_test_p)
pred=np.argmax(predict_test,1)

In [ ]:
df=pd.DataFrame(pred)
df.index.name='ids'
df.columns=['labels']
df.to_csv('final_result.csv',header=True)